In [3]:
import os
import numpy as np
import tensorflow as tf
import torch
from torch.utils.data import DataLoader
import scipy.io.wavfile as wav
import shutil

# --- 1. SETUP DUMMY DATA -----------------------------------
print("[1] Generating Dummy Data...")
DUMMY_DIR = "dummy_dataset"
if os.path.exists(DUMMY_DIR):
    shutil.rmtree(DUMMY_DIR)
os.makedirs(DUMMY_DIR)

# Kita buat 4 Speaker (Class 0, 1, 2, 3), masing-masing 5 file audio
dummy_paths = []
dummy_labels = []
n_speakers = 4
n_files_per_speaker = 5

for spk_id in range(n_speakers):
    for i in range(n_files_per_speaker):
        fname = os.path.join(DUMMY_DIR, f"spk{spk_id}_{i}.wav")
        # Buat audio sine wave sederhana
        sr = 16000
        t = np.linspace(0, 4, sr * 4) # 4 detik
        y = 0.5 * np.sin(2 * np.pi * (220 + spk_id*50) * t)
        wav.write(fname, sr, y.astype(np.float32))
        
        dummy_paths.append(fname)
        dummy_labels.append(spk_id)

print(f"    Total files: {len(dummy_paths)}")
print(f"    Labels sample: {dummy_labels[:10]}...")

# Config sederhana
config = {
    'sample_rate': 16000,
    'target_duration': 3,
    'feature_extractors': ['gfcc'] # Asumsi Anda punya fungsi ini atau mock
}

2026-02-03 17:25:57.319521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[1] Generating Dummy Data...
    Total files: 20
    Labels sample: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]...


In [7]:
dummy_labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]

In [9]:
from src.utils import (
    collect_filepaths_and_labels,
    BalancedBatchScheduler,
    TorchBalancedSampler
)

P, K = 2, 2 
scheduler = BalancedBatchScheduler(dummy_labels, n_classes=P, n_samples=K)

In [10]:
scheduler

In [11]:
for i, batch_indices in enumerate(scheduler):
    batch_labels = [dummy_labels[idx] for idx in batch_indices]
    print(f"    Batch {i} Indices: {batch_indices}")
    print(f"    Batch {i} Labels : {batch_labels}")

    Batch 0 Indices: [np.int64(19), np.int64(18), np.int64(14), np.int64(12)]
    Batch 0 Labels : [3, 3, 2, 2]
    Batch 1 Indices: [np.int64(12), np.int64(13), np.int64(19), np.int64(15)]
    Batch 1 Labels : [2, 2, 3, 3]
    Batch 2 Indices: [np.int64(17), np.int64(17), np.int64(14), np.int64(12)]
    Batch 2 Labels : [3, 3, 2, 2]
    Batch 3 Indices: [np.int64(2), np.int64(1), np.int64(8), np.int64(7)]
    Batch 3 Labels : [0, 0, 1, 1]


In [13]:
batch_labels[0]

0

In [14]:
# --- 2. TEST SAMPLER LOGIC ---------------------------------
print("\n[2] Testing BalancedBatchScheduler (Logic Core)...")
# Skenario DML: 2 Speaker per batch, 2 Utterance per speaker
# Total batch size = 4
P, K = 2, 2 
scheduler = BalancedBatchScheduler(dummy_labels, n_classes=P, n_samples=K)

print(f"    Skenario: P={P}, K={K} (Batch Size {P*K})")
for i, batch_indices in enumerate(scheduler):
    batch_labels = [dummy_labels[idx] for idx in batch_indices]
    print(f"    Batch {i} Indices: {batch_indices}")
    print(f"    Batch {i} Labels : {batch_labels}")
    
    # Validasi Logic
    if len(set(batch_labels)) != P:
        print("    [FAIL] Jumlah speaker dalam batch salah!")
    elif batch_labels[0] != batch_labels[1]: # Cek pasangan (K=2)
        print("    [FAIL] Pasangan label tidak berurutan!")
    else:
        print("    [PASS] Logika Sampler OK.")
    break # Cek 1 batch saja


[2] Testing BalancedBatchScheduler (Logic Core)...
    Skenario: P=2, K=2 (Batch Size 4)
    Batch 0 Indices: [np.int64(6), np.int64(7), np.int64(19), np.int64(15)]
    Batch 0 Labels : [1, 1, 3, 3]
    [PASS] Logika Sampler OK.


In [16]:
from src.dataloader import AudioTFDataset, AudioPTDataset

# --- 3. TEST TENSORFLOW PIPELINE ---------------------------
print("\n[3] Testing TensorFlow DML Pipeline...")
try:
    tf_ds = AudioTFDataset(dummy_paths, dummy_labels, config)
    # Panggil method khusus DML
    dml_tf_dataset = tf_ds.get_dml_dataset(n_classes=P, n_samples=K)
    
    for features, labels in dml_tf_dataset.take(1):
        print("    [PASS] Batch Loaded via TensorFlow.")
        print(f"    Label Shape: {labels.shape} (Expect: {P*K})")
        print(f"    Label Data : {labels.numpy()}")
        
        if 'gfcc' in features:
            print(f"    GFCC Shape : {features['gfcc'].shape}") # (Batch, Time, Freq, Ch)
            
        # Cek Tipe Data
        if features['gfcc'].dtype == tf.float32:
            print("    [PASS] Data Type is float32.")
        else:
            print(f"    [FAIL] Data Type is {features['gfcc'].dtype}")

except Exception as e:
    print(f"    [FAIL] Error TF: {e}")
    import traceback
    traceback.print_exc()


[3] Testing TensorFlow DML Pipeline...


2026-02-03 17:58:13.452138: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


    [PASS] Batch Loaded via TensorFlow.
    Label Shape: (4,) (Expect: 4)
    Label Data : [0 0 1 1]
    [FAIL] Error TF: 'gfcc'


Traceback (most recent call last):
  File "/tmp/ipykernel_60233/2279718630.py", line 19, in <module>
    if features['gfcc'].dtype == tf.float32:
       ~~~~~~~~^^^^^^^^
KeyError: 'gfcc'


In [39]:
os.listdir('../data/interim/train')

['id10286',
 'id10278',
 'id10282',
 'id10297',
 'id10277',
 'id10305',
 'id10302',
 'id10292',
 'id10283',
 'id10281',
 'id10280',
 'id10290',
 'id10288',
 'id10301',
 'id10291',
 'id10309',
 'id10271',
 'id10303',
 'id10274',
 'id10299',
 'id10275',
 'id10298',
 'id10279',
 'id10285',
 'id10289',
 'id10307',
 'id10308',
 'id10296',
 'id10293',
 'id10306',
 'id10272',
 'id10294']

In [40]:
len(os.listdir('../data/interim/train'))

32

In [14]:
os.listdir('../data/raw/vox1_test_wav/wav/id10284')

['YN4cTBWM-QE',
 'Uzxv7Axh3Z8',
 'O_rxcwBWTz4',
 'rOcOdev0azY',
 'RNYNkXzY5Hk',
 '7yx9A0yzLYk',
 'EoCPhxtWUOc',
 'Rzpm9hlXV6I']

In [43]:
import os

os.path.basename(os.path.dirname('repositories/ASR/data/interim/train/id10271/_pOfzIZVLFQ_00001.wav'))

'id10271'

In [21]:
os.path.basename(os.path.dirname(os.path.dirname('repositories/ASR/data/raw/vox1_test_wav/wav/id10270/5r0dWxy17C8/00001.wav')))

'id10270'

In [6]:
os.path.basename(os.path.dirname('repositories/ASR/data/raw/vox1_test_wav/wav/id10270/5r0dWxy17C8/00001.wav'))

'5r0dWxy17C8'

In [ ]:
from glob import glob

data_path = "../data/interim/train"

files = glob(os.path.join(data_path, '**', '*.wav'), recursive=True)

In [56]:
files

['../data/interim/val/id10304/CN6JpPYbHCQ_00008.wav',
 '../data/interim/val/id10304/hDBMV_0Vz4E_00016.wav',
 '../data/interim/val/id10304/CN6JpPYbHCQ_00007.wav',
 '../data/interim/val/id10304/HTL8iLI75TY_00009.wav',
 '../data/interim/val/id10304/U5YB7fT0We4_00002.wav',
 '../data/interim/val/id10304/JQtDfEz08aU_00037.wav',
 '../data/interim/val/id10304/U5YB7fT0We4_00001.wav',
 '../data/interim/val/id10304/zV7LlMi0eHo_00010.wav',
 '../data/interim/val/id10304/JQtDfEz08aU_00011.wav',
 '../data/interim/val/id10304/o9bD_E99Jpg_00015.wav',
 '../data/interim/val/id10304/Wab8UZNYhM0_00001.wav',
 '../data/interim/val/id10304/FJOvOooCIvs_00004.wav',
 '../data/interim/val/id10304/qbrGkPrDuoU_00001.wav',
 '../data/interim/val/id10304/JQtDfEz08aU_00002.wav',
 '../data/interim/val/id10304/CaR6c7RO7Dc_00002.wav',
 '../data/interim/val/id10304/jUSC4i_eGHs_00003.wav',
 '../data/interim/val/id10304/zV7LlMi0eHo_00006.wav',
 '../data/interim/val/id10304/7geDDtDmWQU_00006.wav',
 '../data/interim/val/id1030

In [57]:
################################################################
classes = sorted(list(set([os.path.basename(os.path.dirname(f)) for f in files])))
class_to_idx = {c: i for i, c in enumerate(classes)}
labels = [class_to_idx[os.path.basename(os.path.dirname(f))] for f in files]
################################################################

In [58]:
print(classes)
print(len(classes))

['id10270', 'id10271', 'id10272', 'id10273', 'id10274', 'id10275', 'id10276', 'id10277', 'id10278', 'id10279', 'id10280', 'id10281', 'id10282', 'id10283', 'id10284', 'id10285', 'id10286', 'id10287', 'id10288', 'id10289', 'id10290', 'id10291', 'id10292', 'id10293', 'id10294', 'id10295', 'id10296', 'id10297', 'id10298', 'id10299', 'id10300', 'id10301', 'id10302', 'id10303', 'id10304', 'id10305', 'id10306', 'id10307', 'id10308', 'id10309']
40


In [49]:
print(labels)
print(len(labels))

[34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,

In [50]:
class_to_idx

{'id10270': 0,
 'id10271': 1,
 'id10272': 2,
 'id10273': 3,
 'id10274': 4,
 'id10275': 5,
 'id10276': 6,
 'id10277': 7,
 'id10278': 8,
 'id10279': 9,
 'id10280': 10,
 'id10281': 11,
 'id10282': 12,
 'id10283': 13,
 'id10284': 14,
 'id10285': 15,
 'id10286': 16,
 'id10287': 17,
 'id10288': 18,
 'id10289': 19,
 'id10290': 20,
 'id10291': 21,
 'id10292': 22,
 'id10293': 23,
 'id10294': 24,
 'id10295': 25,
 'id10296': 26,
 'id10297': 27,
 'id10298': 28,
 'id10299': 29,
 'id10300': 30,
 'id10301': 31,
 'id10302': 32,
 'id10303': 33,
 'id10304': 34,
 'id10305': 35,
 'id10306': 36,
 'id10307': 37,
 'id10308': 38,
 'id10309': 39}

In [30]:
from .src.dataloader import AudioPTDataset

ds = AudioPTDataset(
        file_paths=files, 
        labels=labels, 
        preprocess=True,
        feature_extractors=feats_to_use,
        target_duration=config.get('max_duration', 3.0)
    )

ImportError: attempted relative import with no known parent package

In [37]:
import os
import shutil
from pathlib import Path
from tqdm.notebook import tqdm

# ================= KONFIGURASI PATH =================
# Sesuaikan path ini dengan lokasi dataset Anda yang sebenarnya
SOURCE_ROOT = Path('../data/raw/vox1_test_wav/wav')
DEST_ROOT = Path('../data/processed/vox1_test_wav/wav')
# ====================================================

def restructure_voxceleb(source_dir, dest_dir):
    # Cek apakah source directory ada
    if not source_dir.exists():
        print(f"Error: Folder sumber tidak ditemukan di {source_dir}")
        return

    # Ambil list semua folder ID Speaker (contoh: id10270, id10271)
    speaker_dirs = [d for d in source_dir.iterdir() if d.is_dir()]
    
    print(f"Ditemukan {len(speaker_dirs)} pembicara. Memulai proses restrukturisasi...")
    
    # Loop untuk setiap speaker
    for speaker_dir in tqdm(speaker_dirs, desc="Processing Speakers"):
        speaker_id = speaker_dir.name
        
        # Buat folder tujuan untuk speaker ini di dalam folder processed
        # ../data/processed/vox1_test_wav/wav/id10270
        target_speaker_path = dest_dir / speaker_id
        target_speaker_path.mkdir(parents=True, exist_ok=True)
        
        # Loop untuk setiap folder sesi di dalam folder speaker (contoh: 5r0dWxy17C8)
        # Inilah folder yang ingin kita hilangkan level-nya
        for session_dir in speaker_dir.iterdir():
            if not session_dir.is_dir():
                continue
                
            session_id = session_dir.name
            
            # Loop semua file wav di dalam folder sesi
            for wav_file in session_dir.glob('*.wav'):
                # FORMAT NAMA BARU: {session_id}_{original_name}.wav
                # Ini PENTING untuk mencegah file dengan nama sama (00001.wav) saling menimpa
                new_filename = f"{session_id}_{wav_file.name}"
                
                # Path tujuan akhir
                destination_file = target_speaker_path / new_filename
                
                # Copy file (gunakan copy2 untuk mempertahankan metadata)
                shutil.copy2(wav_file, destination_file)

    print("\n✅ Proses Selesai!")
    print(f"Dataset tersimpan di: {dest_dir}")

# Jalankan fungsi
restructure_voxceleb(SOURCE_ROOT, DEST_ROOT)

Ditemukan 40 pembicara. Memulai proses restrukturisasi...


Processing Speakers:   0%|          | 0/40 [00:00<?, ?it/s]


✅ Proses Selesai!
Dataset tersimpan di: ../data/processed/vox1_test_wav/wav


In [38]:
import os
import shutil
import random
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# ================= KONFIGURASI =================
# Path dataset yang sudah dirapikan (hasil dari langkah sebelumnya)
SOURCE_PATH = Path('../data/processed/vox1_test_wav/wav')

# Path tujuan penyimpanan
DEST_ROOT = Path('../data/interim')

# Rasio Pembagian (Total harus 1.0)
TRAIN_RATIO = 0.8
VAL_RATIO   = 0.1
TEST_RATIO  = 0.1

SEED = 42  # Agar hasil acakan konsisten setiap dijalankan
# ===============================================

def split_dataset_by_speaker(
        source, dest, 
        train_ratio, val_ratio, test_ratio):
    # 1. Validasi Path
    if not source.exists():
        print(f"❌ Error: Folder sumber tidak ditemukan di {source}")
        return

    # 2. Ambil daftar semua ID Pembicara (Folder)
    speaker_dirs = [d for d in source.iterdir() if d.is_dir()]
    total_speakers = len(speaker_dirs)
    
    if total_speakers == 0:
        print("❌ Error: Tidak ada folder speaker ditemukan.")
        return

    print(f"🔍 Ditemukan total {total_speakers} pembicara.")

    # 3. Acak urutan pembicara
    random.seed(SEED)
    random.shuffle(speaker_dirs)

    # 4. Hitung titik potong (indices)
    train_end = int(total_speakers * train_ratio)
    val_end = train_end + int(total_speakers * val_ratio)

    # 5. Bagi list pembicara
    train_speakers = speaker_dirs[:train_end]
    val_speakers   = speaker_dirs[train_end:val_end]
    test_speakers  = speaker_dirs[val_end:]

    print(f"📊 Pembagian:")
    print(f"   - Train : {len(train_speakers)} speakers")
    print(f"   - Val   : {len(val_speakers)} speakers")
    print(f"   - Test  : {len(test_speakers)} speakers")

    # 6. Fungsi Helper untuk Copy/Move
    def copy_speakers(speaker_list, split_name):
        target_dir = dest / split_name
        target_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📂 Menyalin data ke {split_name}...")
        for speaker_dir in tqdm(speaker_list, desc=f"Copying to {split_name}"):
            # Source: ../processed/id10270
            # Dest:   ../interim/train/id10270
            dest_path = target_dir / speaker_dir.name
            
            # Gunakan copytree untuk menyalin folder beserta isinya
            # dirs_exist_ok=True agar tidak error jika dijalankan ulang
            shutil.copytree(speaker_dir, dest_path, dirs_exist_ok=True)

    # 7. Eksekusi Pemindahan
    copy_speakers(train_speakers, 'train')
    copy_speakers(val_speakers, 'val')
    copy_speakers(test_speakers, 'test')

    print("\n✅ Proses Splitting Selesai!")
    print(f"Data tersimpan di: {dest}")

# Jalankan Fungsi
split_dataset_by_speaker(SOURCE_PATH, DEST_ROOT, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

🔍 Ditemukan total 40 pembicara.
📊 Pembagian:
   - Train : 32 speakers
   - Val   : 4 speakers
   - Test  : 4 speakers

📂 Menyalin data ke train...


Copying to train:   0%|          | 0/32 [00:00<?, ?it/s]


📂 Menyalin data ke val...


Copying to val:   0%|          | 0/4 [00:00<?, ?it/s]


📂 Menyalin data ke test...


Copying to test:   0%|          | 0/4 [00:00<?, ?it/s]


✅ Proses Splitting Selesai!
Data tersimpan di: ../data/interim
